In [33]:
# https://www.tensorflow.org/api_docs/python/tf/compat/v1/estimator/experimental/KMeans

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import io
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow.compat.v1.estimator.experimental import KMeans

In [8]:
# import the training dataset
solana_train = pd.read_csv("../../DL-Photos/solana_train.csv")
solana_train=solana_train['img']

In [12]:
# Parameters
num_steps = 50 # Total steps to train
batch_size = 1024 # The number of samples per batch
k = 25 # The number of clusters
num_classes = 10 # The 10 digits
num_features = 50176 # Each image is  224*224 pixels

In [22]:
X = tf.placeholder(tf.float32, shape=[None, num_features])
Y = tf.placeholder(tf.float32, shape=[None, num_classes])
kmeans = KMeans(num_clusters=k, distance_metric='cosine',use_mini_batch=True)
#import inspect
#inspect.getargspec(KMeans) # This one is to check the arguments that KMeans take

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp7l9q4klr', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [32]:
num_points = 100
dimensions = 2
points = np.random.uniform(0, 1000, [num_points, dimensions])
def input_fn():
  return tf.compat.v1.train.limit_epochs(
      tf.convert_to_tensor(points, dtype=tf.float32), num_epochs=1)
# train
num_iterations = 10
previous_centers = None
for _ in range(0,num_iterations):
  kmeans.train(input_fn)
  cluster_centers = kmeans.cluster_centers()
  if previous_centers is not None:
    print('delta:', cluster_centers - previous_centers)
  previous_centers = cluster_centers
  print('score:', kmeans.score(input_fn))
print('cluster centers:', cluster_centers)



Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
Prefer Dataset.range instead.
Instructions for updating:
Prefer Dataset.range instead.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
dim is deprecated, use axis instead
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp7l9q4klr/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:Calling checkpoint listeners before saving c

"\n# map the input points to their clusters\ncluster_indices = list(kmeans.predict_cluster_index(input_fn))\nfor i, point in enumerate(points):\n  cluster_index = cluster_indices[i]\n  center = cluster_centers[cluster_index]\n  print('point:', point, 'is in cluster', cluster_index, 'centered at', center)"

In [34]:
# map the input points to their clusters
cluster_indices = list(kmeans.predict_cluster_index(input_fn))
for i, point in enumerate(points):
  cluster_index = cluster_indices[i]
  center = cluster_centers[cluster_index]
  print('point:', point, 'is in cluster', cluster_index, 'centered at', center)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp7l9q4klr/model.ckpt-19
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
point: [621.55610169 722.1381733 ] is in cluster 15 centered at [0.65565795 0.7549455 ]
point: [107.18607389 422.1878142 ] is in cluster 19 centered at [0.25321752 0.9672021 ]
point: [355.75180801 909.37888142] is in cluster 3 centered at [0.34821314 0.9364793 ]
point: [341.35118793 311.97959609] is in cluster 0 centered at [0.7423189 0.6697797]
point: [911.35549912 177.75754448] is in cluster 24 centered at [0.98168075 0.19026121]
point: [421.04467283 616.09531555] is in cluster 1 centered at [0.58589923 0.80981004]
point: [144.14735758 784.41145069] is in cluster 21 centered at [0.15960851 0.9869174 ]
point: [730.31808122 814.86041083] is in cluster 15 centered at [0.65565795 0.7549455 ]
point: [525.90352805 990.01801188] 